In [1]:
param_x_api_key = '2BeBXg69osbuG45Rjtv1wLEnHL3nuT3WPQQK1EaYHT4PutPkDrXB'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part_2.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                data = '''{"getArticleLinkedAllLinkingTarget4":
                {"provider":40000,
                 "articleId":''' + input_.loc[a, 'Article_Id'] + ''',
                 "lang":"en",
                 "articleCountry":"DE",
                 "country":"DE",
                 "countryGroupFlag":false,
                 "linkingTargetManuId":''' + input_.loc[a, 'Manufactuer_Id'] + ''',
                 "linkingTargetType":"VOLB",
                 "withMainArticles":false}}'''

                headers = UA.get_User_Agent_Requests()
                headers['X-Api-Key'] = param_x_api_key

                resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                                     data=data,
                                     headers=headers,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                if json_data['status'] == 200 and json_data['data']['array'][0]['articleLinkages'] == '':
                    crawler_status = 'ok'
                    
                    break
                
                # = = = = = = = = = = = = = = =
                
                list_aiti = [str(array['articleLinkId']) + '-' + str(array['linkingTargetId']) for array in json_data['data']['array'][0]['articleLinkages']['array']]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Article_Id': input_.loc[a, 'Article_Id'],
                                        'AITI': list_aiti})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Article_Id': input_.loc[a, 'Article_Id'],
                                     'Manufactuer_Id': input_.loc[a, 'Manufactuer_Id']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Article_Id'] + ' - ' + input_.loc[a, 'Manufactuer_Id'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Article_Id', 'AITI'], ascending=[True, True]).reset_index(drop=True)
output_correct.to_excel('./target-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Article_Id', 'Manufactuer_Id'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./target_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：775

[ok] - 1200870258 - 93
[尝试次数：1] - [剩余数量：755] - [当前时间：14:22:27]

[ok] - 1095800452 - 184
[尝试次数：1] - [剩余数量：754] - [当前时间：14:22:27]

[ok] - 1152765695 - 183
[尝试次数：1] - [剩余数量：753] - [当前时间：14:22:27]

[ok] - 1095902317 - 120
[尝试次数：1] - [剩余数量：752] - [当前时间：14:22:28]

[ok] - 1200870258 - 80
[尝试次数：1] - [剩余数量：751] - [当前时间：14:22:28]

[ok] - 1152686457 - 72
[尝试次数：1] - [剩余数量：750] - [当前时间：14:22:28]

[ok] - 1220272530 - 36
[尝试次数：1] - [剩余数量：749] - [当前时间：14:22:28]

[ok] - 1152784537 - 183
[尝试次数：1] - [剩余数量：748] - [当前时间：14:22:28]

[ok] - 1220272530 - 5470
[尝试次数：1] - [剩余数量：747] - [当前时间：14:22:28]

[ok] - 1095902317 - 5470
[尝试次数：1] - [剩余数量：746] - [当前时间：14:22:28]

[ok] - 1095867956 - 84
[尝试次数：1] - [剩余数量：745] - [当前时间：14:22:28]

[ok] - 1226075431 - 753
[尝试次数：1] - [剩余数量：744] - [当前时间：14:22:28]

[ok] - 1226075431 - 84
[尝试次数：1] - [剩余数量：743] - [当前时间：14:22:28]

[ok] - 1095867956 - 753
[尝试次数：1] - [剩余数量：742] - [当前时间：14:22:29]

[ok] - 1254034619 - 36
[尝试次数：1] - [剩余数量：741] - [当前时间：14:22:29]

[ok] - 1217959742 - 1

[ok] - 1380888514 - 1820
[尝试次数：1] - [剩余数量：627] - [当前时间：14:22:44]

[ok] - 1373708371 - 84
[尝试次数：4] - [剩余数量：626] - [当前时间：14:22:44]

[ok] - 1380492438 - 80
[尝试次数：2] - [剩余数量：625] - [当前时间：14:22:44]

[ok] - 1373708339 - 84
[尝试次数：4] - [剩余数量：624] - [当前时间：14:22:44]

[ok] - 1388919874 - 106
[尝试次数：3] - [剩余数量：623] - [当前时间：14:22:45]

[ok] - 1388919874 - 5
[尝试次数：1] - [剩余数量：622] - [当前时间：14:22:45]

[ok] - 1407091521 - 55
[尝试次数：2] - [剩余数量：621] - [当前时间：14:22:45]

[ok] - 1380888514 - 5470
[尝试次数：1] - [剩余数量：620] - [当前时间：14:22:46]

[ok] - 1388919874 - 121
[尝试次数：2] - [剩余数量：619] - [当前时间：14:22:46]

[ok] - 1397126948 - 111
[尝试次数：3] - [剩余数量：618] - [当前时间：14:22:46]

[ok] - 1408315556 - 5470
[尝试次数：1] - [剩余数量：617] - [当前时间：14:22:46]

[ok] - 1417544955 - 77
[尝试次数：1] - [剩余数量：616] - [当前时间：14:22:47]

[ok] - 1417533775 - 77
[尝试次数：1] - [剩余数量：615] - [当前时间：14:22:47]

[ok] - 1417593590 - 77
[尝试次数：3] - [剩余数量：614] - [当前时间：14:22:47]

[ok] - 1408243002 - 93
[尝试次数：3] - [剩余数量：613] - [当前时间：14:22:47]

[ok] - 1408315556 - 36
[尝试次数：1] 

[ok] - 44765004 - 162
[尝试次数：1] - [剩余数量：496] - [当前时间：14:23:18]

[ok] - 44825451 - 753
[尝试次数：1] - [剩余数量：495] - [当前时间：14:23:18]

[ok] - 44825451 - 117
[尝试次数：1] - [剩余数量：494] - [当前时间：14:23:18]

[ok] - 44765004 - 69
[尝试次数：1] - [剩余数量：493] - [当前时间：14:23:19]

[ok] - 44400558 - 36
[尝试次数：1] - [剩余数量：492] - [当前时间：14:23:19]

[ok] - 44950489 - 74
[尝试次数：1] - [剩余数量：491] - [当前时间：14:23:20]

[ok] - 44825451 - 84
[尝试次数：1] - [剩余数量：490] - [当前时间：14:23:21]

[ok] - 44970156 - 103
[尝试次数：1] - [剩余数量：489] - [当前时间：14:23:21]

[ok] - 44958301 - 776
[尝试次数：1] - [剩余数量：488] - [当前时间：14:23:21]

[ok] - 44958301 - 36
[尝试次数：1] - [剩余数量：487] - [当前时间：14:23:22]

[ok] - 44947636 - 74
[尝试次数：1] - [剩余数量：486] - [当前时间：14:23:22]

[ok] - 44977167 - 74
[尝试次数：1] - [剩余数量：485] - [当前时间：14:23:22]

[ok] - 45083238 - 74
[尝试次数：1] - [剩余数量：484] - [当前时间：14:23:23]

[ok] - 45083247 - 74
[尝试次数：1] - [剩余数量：483] - [当前时间：14:23:23]

[ok] - 45127529 - 16
[尝试次数：1] - [剩余数量：482] - [当前时间：14:23:24]

[ok] - 44810390 - 74
[尝试次数：1] - [剩余数量：481] - [当前时间：14:23:24]

[ok

[ok] - 46606651 - 776
[尝试次数：1] - [剩余数量：364] - [当前时间：14:24:00]

[ok] - 1344230263 - 36
[尝试次数：7] - [剩余数量：363] - [当前时间：14:24:01]

[ok] - 46661489 - 121
[尝试次数：1] - [剩余数量：362] - [当前时间：14:24:01]

[ok] - 46661489 - 104
[尝试次数：1] - [剩余数量：361] - [当前时间：14:24:01]

[ok] - 46658067 - 120
[尝试次数：1] - [剩余数量：360] - [当前时间：14:24:01]

[ok] - 46661566 - 121
[尝试次数：1] - [剩余数量：359] - [当前时间：14:24:01]

[ok] - 46657588 - 5
[尝试次数：1] - [剩余数量：358] - [当前时间：14:24:02]

[ok] - 46605733 - 64
[尝试次数：1] - [剩余数量：357] - [当前时间：14:24:02]

[ok] - 1380888514 - 56
[尝试次数：4] - [剩余数量：356] - [当前时间：14:24:02]

[ok] - 46788811 - 88
[尝试次数：1] - [剩余数量：355] - [当前时间：14:24:02]

[ok] - 46821227 - 35
[尝试次数：1] - [剩余数量：354] - [当前时间：14:24:02]

[ok] - 46662459 - 93
[尝试次数：1] - [剩余数量：353] - [当前时间：14:24:03]

[ok] - 46837054 - 35
[尝试次数：1] - [剩余数量：352] - [当前时间：14:24:03]

[ok] - 46610407 - 120
[尝试次数：1] - [剩余数量：351] - [当前时间：14:24:03]

[ok] - 46657588 - 106
[尝试次数：1] - [剩余数量：350] - [当前时间：14:24:03]

[ok] - 46837359 - 35
[尝试次数：1] - [剩余数量：349] - [当前时间：14:24:03]

[ok] - 47838944 - 20
[尝试次数：1] - [剩余数量：230] - [当前时间：14:24:31]

[ok] - 47917514 - 74
[尝试次数：1] - [剩余数量：229] - [当前时间：14:24:32]

[ok] - 47923865 - 93
[尝试次数：1] - [剩余数量：228] - [当前时间：14:24:32]

[ok] - 47935932 - 80
[尝试次数：1] - [剩余数量：227] - [当前时间：14:24:32]

[ok] - 47933667 - 80
[尝试次数：1] - [剩余数量：226] - [当前时间：14:24:32]

[ok] - 47923865 - 139
[尝试次数：1] - [剩余数量：225] - [当前时间：14:24:33]

[ok] - 47933667 - 93
[尝试次数：3] - [剩余数量：224] - [当前时间：14:24:33]

[ok] - 47936003 - 139
[尝试次数：1] - [剩余数量：223] - [当前时间：14:24:33]

[ok] - 47935932 - 93
[尝试次数：2] - [剩余数量：222] - [当前时间：14:24:33]

[ok] - 47935577 - 93
[尝试次数：2] - [剩余数量：221] - [当前时间：14:24:33]

[ok] - 47923865 - 80
[尝试次数：4] - [剩余数量：220] - [当前时间：14:24:34]

[ok] - 47936003 - 93
[尝试次数：2] - [剩余数量：219] - [当前时间：14:24:34]

[ok] - 47937915 - 21
[尝试次数：2] - [剩余数量：218] - [当前时间：14:24:34]

[ok] - 47944747 - 74
[尝试次数：2] - [剩余数量：217] - [当前时间：14:24:34]

[ok] - 47936003 - 74
[尝试次数：1] - [剩余数量：216] - [当前时间：14:24:34]

[ok] - 47946076 - 36
[尝试次数：1] - [剩余数量：215] - [当前时间：14:24:34]

[ok] -

[ok] - 49031841 - 104
[尝试次数：1] - [剩余数量：95] - [当前时间：14:25:01]

[ok] - 48879779 - 35
[尝试次数：6] - [剩余数量：94] - [当前时间：14:25:01]

[ok] - 49031841 - 121
[尝试次数：1] - [剩余数量：93] - [当前时间：14:25:01]

[ok] - 49021995 - 35
[尝试次数：3] - [剩余数量：92] - [当前时间：14:25:01]

[ok] - 49031841 - 5
[尝试次数：1] - [剩余数量：91] - [当前时间：14:25:02]

[ok] - 49033440 - 121
[尝试次数：2] - [剩余数量：90] - [当前时间：14:25:02]

[ok] - 49031841 - 3035
[尝试次数：3] - [剩余数量：89] - [当前时间：14:25:02]

[ok] - 49031841 - 4896
[尝试次数：4] - [剩余数量：88] - [当前时间：14:25:02]

[ok] - 49033577 - 104
[尝试次数：2] - [剩余数量：87] - [当前时间：14:25:03]

[ok] - 49033440 - 5
[尝试次数：4] - [剩余数量：86] - [当前时间：14:25:03]

[ok] - 49031841 - 106
[尝试次数：5] - [剩余数量：85] - [当前时间：14:25:03]

[ok] - 49033440 - 3035
[尝试次数：5] - [剩余数量：84] - [当前时间：14:25:03]

[ok] - 49033577 - 121
[尝试次数：1] - [剩余数量：83] - [当前时间：14:25:04]

[ok] - 49033440 - 104
[尝试次数：2] - [剩余数量：82] - [当前时间：14:25:04]

[ok] - 49033577 - 5
[尝试次数：1] - [剩余数量：81] - [当前时间：14:25:04]

[ok] - 49033577 - 2859
[尝试次数：1] - [剩余数量：80] - [当前时间：14:25:04]

[ok] - 49033